[View in Colaboratory](https://colab.research.google.com/github/agerk/machineLearning/blob/master/tf_Dataset_tutorial_part_1.ipynb)

In [4]:
# Install latest version of tensorflow
!pip install tensorflow==1.8.0

    100% |████████████████████████████████| 49.1MB 989kB/s 
    100% |████████████████████████████████| 3.1MB 10.5MB/s 
  Found existing installation: tensorboard 1.7.0
    Uninstalling tensorboard-1.7.0:
      Successfully uninstalled tensorboard-1.7.0
  Found existing installation: tensorflow 1.7.0
    Uninstalling tensorflow-1.7.0:
      Successfully uninstalled tensorflow-1.7.0


<h3> Install Google Drive</h3>

In [0]:
# Code to read csv file into colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


<h2> Intro to TensorfFow Dataset and Estimator</h2>

- this tutorial follows the the tutorial series on tf.Dataset and tf.Estimator by Google
- example model is the Iris flower classification problem

In [6]:
# Load liberary
import tensorflow as tf
import pandas as pd
import os
print(tf.__version__)

1.8.0


<h3> Download dataset</h3>

In [0]:
# Download dataset
# Get the file from Drive

# get id of train  and data from Google Drive
train = drive.CreateFile({'id':'1da4BRxdb5CoAsTs2omowTLQnZGEVhvjo'})   # replace the id with id of file you want to access
train.GetContentFile('iris_training.csv')  

test = drive.CreateFile({'id':'1ThMeLns4SyPwZTPivUOheM7nKq5RF2AI'})   # replace the id with id of file you want to access
test.GetContentFile('iris_test.csv')  

#3. Read file as panda dataframe
# iris_training = pd.read_csv('iris_training.csv')
# iris_test = pd.read_csv('iris_test.csv') 

In [8]:
train

GoogleDriveFile({'id': '1da4BRxdb5CoAsTs2omowTLQnZGEVhvjo', 'kind': 'drive#file', 'etag': '"qu1qQsDCUNdGLg7FCvF6ozYbuj8/MTUyNjgzOTI2Nzk5NA"', 'selfLink': 'https://www.googleapis.com/drive/v2/files/1da4BRxdb5CoAsTs2omowTLQnZGEVhvjo', 'webContentLink': 'https://drive.google.com/uc?id=1da4BRxdb5CoAsTs2omowTLQnZGEVhvjo&export=download', 'alternateLink': 'https://drive.google.com/file/d/1da4BRxdb5CoAsTs2omowTLQnZGEVhvjo/view?usp=drivesdk', 'embedLink': 'https://drive.google.com/file/d/1da4BRxdb5CoAsTs2omowTLQnZGEVhvjo/preview?usp=drivesdk', 'iconLink': 'https://drive-thirdparty.googleusercontent.com/16/type/text/csv', 'title': 'iris_training.csv', 'mimeType': 'text/csv', 'labels': {'starred': False, 'hidden': False, 'trashed': False, 'restricted': False, 'viewed': True}, 'createdDate': '2018-05-20T17:31:06.235Z', 'modifiedDate': '2018-05-20T18:01:07.994Z', 'modifiedByMeDate': '2018-05-20T18:01:07.994Z', 'lastViewedByMeDate': '2018-05-20T17:31:06.235Z', 'markedViewedByMeDate': '1970-01-01T00

<h2> Create features for Deep Neural Network Classifier</h2>

Model Atchitecture:

1.  Input to model 
  - Sepal length
  - Sepal width
  - Petal length
  - Petal width
  - lable (0 for Setosa, 1 for Versicolor, and 2 for Virginica)

1.   Two hidden layers with 10 neurons each
2.   Outpit prediction is probability of Iris type



<h2> Preprocessing using tf.Dataset</h2>

Its the new way of creating input pipeline for Tensorflow-- better performance than f*eed_dict *and *queue-based* pipelines

**Dataset**: Consist of the follwoing subclasses 
1.  TextLineDataset: reads lines from text files
1.  TFRecordDataset: reads records from disc
2.   FixedLengthRecordDataset: reads fixed size records from binary files

and **Iterator** instance: provides access to dataset elemenst during train, eval and serving. 

In [0]:
!mkdir -p drive
# !google-drive-ocamlfuse drive

In [10]:
ls

datalab/  drive/  iris_test.csv  iris_training.csv


In [0]:
# Iris dataset
# it has 120 example
# 4 features -- sepal length and width, petal length and width
# target variale-- flower type (setosa, versicolor and virginica)

# Fetch and store dataset
PATH = "drive/"
# PATH_DATASET = PATH + os.sep + dataset
FILE_TRAIN = "iris_training.csv"
FILE_TEST = "iris_test.csv"
# URL_TRAIN = "http://download.tensorflow.org/data/iris_training.csv"
# URL_TEST = "http://download.tensorflow.org/data/iris_test.csv"

# def download_dataset(url, file):
#   if not os.path.exists(PATH_DATASET):
#     os.mkdirs(PATH_DATASET)
#   if not os.path.exist(file):
#     data = request.urlopen(url).read()
#     with open(file, "wb") as f:
#       f.write(data)
#       f.close()
# download_dataset(URL_TRAIN, FILE_TRAIN)
# download_dataset(URL_TRAIN, FILE_TRAIN)

# Data representation
feature_names = [
    'SepalLength',
    'SepalWidth',
    'PetalLength',
    'PetalWidth']

# for training, we will need our a function that reads raw input files and reurn feature and label data.
# tf.Estimator requires a function which can accomplish that

# implimentation of data pipeline using Dataset API
def decode_csv(line):
  """
  Args: a line of sample from csv
  Returns: dict with features and label
  """
  parsed_line = tf.decode_csv(line, [[0.], [0.], [0.], [0.], [0]])    # split each line in the dataset. Lable should be integer [0]
  label = parsed_line[-1]    # last element is the flower label (class) 
  del parsed_line[-1]    # delete last elment
  features = parsed_line    # Everythinf minus label
  return dict(zip(feature_names, features)), label

def input_fn(file_path, perform_shuffle=False, repeat_count=1):
  """
  Args: Input file for train or test, shffle, num of times to iterate over the records. 
        for example if repeat_count=1, then record is read only once
  
  Returns: list of features and batch labels for bactch training  
  """
  dataset = (tf.data.TextLineDataset(file_path)    #read text file 
            .skip(1)    # skip column header
            .map(decode_csv))    # Apply element-wise transformation 
#   print(dataset)
  if perform_shuffle:
    dataset = dataset.shuffle(buffer_size=256)    # randomize dataset order using a window of 256 elements
  dataset = dataset.repeat(repeat_count)    # repeat dataset numrepear_count times
  dataset = dataset.batch(32)    # read 32 sample randomly
  iterator = dataset.make_one_shot_iterator()    # iterat throught a batch
  batch_features, batch_labels = iterator.get_next()    # get feature and labels 
  return batch_features, batch_labels

In [12]:
# print out first batch
next_batch = input_fn(FILE_TRAIN, True)
# retrieving and printing one batch of data
with tf.Session() as sess:
  first_batch = sess.run(next_batch)
print(first_batch)

<MapDataset shapes: ({SepalLength: (), SepalWidth: (), PetalLength: (), PetalWidth: ()}, ()), types: ({SepalLength: tf.float32, SepalWidth: tf.float32, PetalLength: tf.float32, PetalWidth: tf.float32}, tf.int32)>
({'SepalLength': array([7. , 5.7, 5.1, 5.3, 5. , 4.6, 5.1, 4.8, 6.8, 6. , 6.7, 5.6, 5. ,
       4.9, 4.9, 7.2, 6.2, 6.9, 7.7, 5.7, 6.5, 5.2, 5. , 5.9, 6.9, 7.9,
       4.6, 6.3, 4.8, 5. , 5.4, 4.9], dtype=float32), 'SepalWidth': array([3.2, 3.8, 3.5, 3.7, 3. , 3.4, 3.8, 3.4, 3. , 2.7, 3.1, 2.5, 3.5,
       2.4, 3. , 3. , 3.4, 3.1, 2.6, 4.4, 3. , 2.7, 3.4, 3.2, 3.1, 3.8,
       3.1, 2.5, 3. , 3.3, 3. , 3.1], dtype=float32), 'PetalLength': array([4.7, 1.7, 1.4, 1.5, 1.6, 1.4, 1.5, 1.6, 5.5, 5.1, 5.6, 3.9, 1.6,
       3.3, 1.4, 5.8, 5.4, 5.1, 6.9, 1.5, 5.2, 3.9, 1.5, 4.8, 4.9, 6.4,
       1.5, 5. , 1.4, 1.4, 4.5, 1.5], dtype=float32), 'PetalWidth': array([1.4, 0.3, 0.3, 0.2, 0.2, 0.3, 0.3, 0.2, 2.1, 1.6, 2.4, 1.1, 0.6,
       1. , 0.2, 1.6, 2.3, 2.3, 2.3, 0.4, 2. , 1.4, 0.2, 1.8,

<h2> tf.Estimator</h2>

High level API that reduces boler plate codes that were necessary to transform data pipeline to tensorflow

It has two possible ways to build model using **Estimator**
- Pre-made Estimator:  canned estimator for specific type of model. It includes
 - DNN classifier and 
 - DNN regressor
- Estimator base class: customemizable estimator

<h3> Model training and evaluation </h3>

- more on tf.feature_column https://developers.googleblog.com/2017/11/introducing-tensorflow-feature-columns.html

In [13]:
# Convert feature_columns to numeric columns which specifies input to the mode
# feature column serve as a bridge between input data and model 
# All features are numeric-- use tf.feature_numeric.numeric_columm use defual dtype-- tf.float32
# Defual numeric column creates a single value(scalar)
feature_columns = [tf.feature_column.numeric_column(col) for col in feature_names]

# Create a deep neural regression classifier using DNNClassifier pre-made estimator
model = tf.estimator.DNNClassifier(
     feature_columns=feature_columns,
     hidden_units = [10, 10],  # two layers with 10 neurons
     n_classes = 3,
     model_dir=PATH)   # path to checkpoints

# Train model for 10 iterations
model.train(input_fn = lambda: input_fn(FILE_TRAIN, True, 10))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'drive/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fb1c47f2e80>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
<MapDataset shapes: ({SepalLength: (), SepalWidth: (), PetalLength: (), PetalWidth: ()}, ()), types: ({SepalLength: tf.float32, SepalWidth: tf.float32, PetalLength: tf.float32, PetalWidth: tf.float32}, tf.int32)>
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Grap

In [14]:
# Model evaluation 
# run model against test data
# return will contain evaluation metrics: accuracy, average loss
model_eval = model.evaluate(
  input_fn = lambda: input_fn(FILE_TEST, False, 4))
print("Evaluation results")
for key in model_eval:
  print("  {} is: {}".format(key, model_eval[key]))

<MapDataset shapes: ({SepalLength: (), SepalWidth: (), PetalLength: (), PetalWidth: ()}, ()), types: ({SepalLength: tf.float32, SepalWidth: tf.float32, PetalLength: tf.float32, PetalWidth: tf.float32}, tf.int32)>
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-05-21-15:45:16
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from drive/model.ckpt-38
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-05-21-15:45:17
INFO:tensorflow:Saving dict for global step 38: accuracy = 0.3, average_loss = 1.0589244, global_step = 38, loss = 31.767735
Evaluation results
  accuracy is: 0.30000001192092896
  average_loss is: 1.0589244365692139
  loss is: 31.76773452758789
  global_step is: 38


<h3>Making Predictions Using Our Trained Model</h3>

In [15]:
# predict the type of Iris flowers
prediction_iris = model.predict(input_fn=lambda: input_fn(FILE_TEST, False, 1))
print("prediction on test data")
for prediction in prediction_iris:
  # will print predicted class, i.e: 0, 1, or 2 if preiction is 
  # Iris Setosa, Vericolor, Virginica
  print(prediction["class_ids"][0])

prediction on test data
<MapDataset shapes: ({SepalLength: (), SepalWidth: (), PetalLength: (), PetalWidth: ()}, ()), types: ({SepalLength: tf.float32, SepalWidth: tf.float32, PetalLength: tf.float32, PetalWidth: tf.float32}, tf.int32)>
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from drive/model.ckpt-38
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
1
2
2
2
2
2
2
2
2
2
2
2
2
2


<h3> Making prediction on data in memory</h3>
  
 this does not change the model configuration, but dataset API to use a memory structure

In [16]:
# create a dataset for prediction
prediction_input = [[5.9, 3.0, 4.2, 1.5],  # -> 1, Iris Versicolor
                    [6.9, 3.1, 5.4, 2.1],  # -> 2, Iris Virginica
                    [5.1, 3.3, 1.7, 0.5]]  # -> 0, Iris Sentosa

def new_input_fn():
  def decode(x):
    x = tf.split(x, 4)    # split into four feature columns
    return dict(zip(feature_names, x))    # build dictionary of features
  
  
  dataset = tf.data.Dataset.from_tensor_slices(prediction_input)
  dataset = dataset.map(decode)
  iterator = dataset.make_one_shot_iterator()
  next_feature_batch = iterator.get_next()
  return next_feature_batch, None   # in predition no label, since we are trying to predict labels based on input

# predicti Iris flower type of all our prediction_input 
predict_memory = model.predict(input_fn=new_input_fn)

# print results
print("Prediction on memory")
for idx, prediction in enumerate(predict_memory):
  type = prediction["class_ids"][0]   # get predicted class index
  if type == 0:
    print(" I think: {}, is Iris Setosa".format(prediction_input[idx]))
  if type == 1:
    print(" I think: {}, is Iris Versicolor".format(prediction_input[idx]))
  if type == 0:
    print(" I think: {}, is Iris Virginica".format(prediction_input[idx]))
  

Prediction on memory
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from drive/model.ckpt-38
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


*This tutorial is inspired by this blog *https://developers.googleblog.com/2017/09/introducing-tensorflow-datasets.html

I strongly recomend to read through to better understand the mechanics of tf.Dataset